# Plastic glacier profiles on idealized bed

In [ ]:
import chakra
import chakra_sermeq
import matplotlib.pyplot as plt
import numpy as np

### Set up standard idealized bed profile

In [ ]:
# Bassis & Ultee bed profile
fls = chakra.bu_tidewater_bed()

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 5))
x = fls[-1].dis_on_line * fls[-1].dx_meter / 1000
ax.hlines(0, 0, 60, color='C0')
ax.plot(x, fls[-1].bed_h, color='k')
ax.set(ylim=(-350, 800), xlabel=('Distance along flowline [km]'), ylabel=('Altitude [m]'));
plt.show()

In [ ]:
len(fls[-1].bed_h)

### Produce a SERMeQ-plastic surface profile

For now, we are going to call a special class `chakra_sermeq.PlasticGlacier`.  Its function `plastic_profile` will produce a basic analytical surface profile for a quasi-steady-state plastic calving glacier.  This function was superseded by some fancier functionality in later SERMeQ, and we'll probably have to rewrite it again.  But it should do the job to start.

In [ ]:
max(fls[-1].bed_h)

We have to flip the coordinates of x to x_nondim to give SERMeQ the coordinate system it expects.  This won't work to hand off between OGGM/SERMeQ, so we'll need to rework `PlasticGlacier.plastic_profile`

In [ ]:
g = chakra_sermeq.PlasticGlacier(yield_strength=150e3)
x_nondim = x[::-1]/(chakra_sermeq.L0/1000) # x is in km, not m
bed_nondim = fls[-1].bed_h/chakra_sermeq.H0
g.set_bed_function(x_nondim, bed_nondim)

init_point = 20000/chakra_sermeq.L0 # initialize 15 km up from end of flowline
s = g.plastic_profile(Bfunction=g.bingham_const,
                  startpoint=init_point, endpoint=max(x_nondim),
                  hinit=g.balance_thickness(g.bed_function(init_point), g.bingham_const())
                     )

In [ ]:
g.bed_function(6.0)

We need to translate from nondimensional to dimensional coordinates, and flip x-axis back, to display on same plot as OGGM.

In [ ]:
xs_dimensional = 10*np.array(s[0]) # multiply by 10, or L0/1000, because original units km
bs_dimensional = chakra_sermeq.H0*np.array(s[2])
ss_dimensional = chakra_sermeq.H0*np.array(s[1])

In [ ]:
min(xs_dimensional)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 5))
ax.plot(xs_dimensional[::-1], bs_dimensional[::-1], color='k') # bed topography
ax.plot(xs_dimensional[::-1], ss_dimensional[::-1], color='b') # glacier surface
ax.plot(x, fls[-1].bed_h, color='k', ls='-.') # topo from ideal OGGM
ax.set(ylim=(-350, 800), xlabel=('Distance along flowline [km]'), ylabel=('Altitude [m]'));

There is a coordinate problem here--relatively easy but fiddly to fix.  Will tackle it tomorrow, but you can play in the meantime!